## **Парсим сайт**

1.   Перебираем ссылки, ищем рабочие
2.   Из рабочих ссылок достаём текст задачи и темы задачи
3.   Приводим в нормальный вид полученные данные

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 1.9 MB/s eta 0:00:00


Достаем рабочие ссылки

In [ ]:
# Создаем пустой список для хранения рабочих ссылок
working_links = []

# Цикл для перебора чисел от 0 до 8000
for num in range(0000, 8000):
    # Создаем ссылку с числом формата "%04d"
    url = f"https://www.iloveeconomics.ru/z/{num:04d}"

    # Отправляем GET-запрос к ссылке
    response = requests.get(url)

    # Проверяем успешность запроса
    if response.status_code == 200:
            # Добавляем рабочую ссылку в список
            working_links.append(url)

    # Выводим текущий номер и рабочее количество ссылок
    print(f"Processed: {num}, Working Links: {len(working_links)}", end="\r")

парсим по готовому набору ссылок

In [2]:
with open('url_list.txt', "r") as file:
    lines = file.readlines()
lines = [line.strip() for line in lines]

# Создаем пустой DataFrame
df = pd.DataFrame(columns=["Task", "Theme"])

# Цикл для перебора url
for url in lines:

    # Отправляем GET-запрос к ссылке
    response = requests.get(url)

    # Проверяем успешность запроса
    if response.status_code == 200:
        # Создаем объект BeautifulSoup для парсинга страницы
        soup = BeautifulSoup(response.content, "html.parser")

        # Находим все элементы с классом "fix_width"
        task_items = soup.find_all(class_="fix_width")

        # Если найдены элементы задачи
        if task_items:
            # Создаем пустые списки для хранения текста задачи и их тем
            tasks = []
            themes = []

            # Итерируемся по каждому элементу задачи
            for task_item in task_items:
                # Находим текст задачи, находящийся в теге "div" с классом "tex2jax"
                task_text = task_item.find(class_="tex2jax")
                tasks.append(task_text)

                # Находим тему задачи, находящуюся в теге "a" с классом "field-item even"
                theme_text = task_item.find(class_="field-item even")
                themes.append(theme_text)

            # Создаем временный DataFrame для текущей ссылки
            temp_df = pd.DataFrame({"Task": tasks, "Theme": themes})

            # Добавляем временный DataFrame в общий DataFrame
            df = pd.concat([df, temp_df], ignore_index=True)

df.dropna(inplace=True)
df.reset_index(drop= True , inplace= True)

In [4]:
theme = pd.read_excel('theme.xlsx')
df.Task = df.Task.astype('str')
df.Theme = df.Theme.astype('str')
# Удаление скобок <> и их содержимого
df["Task"] = df["Task"].apply(lambda x: re.sub(r"<.*?>", "", x))
df["Theme"] = df["Theme"].apply(lambda x: re.sub(r"<.*?>", "", x))
# Удаление символов отступа и абзаца
df["Task"] = df["Task"].apply(lambda x: x.replace("\n", "").replace("\r", ""))

## **Подготовка данных для модели**

In [ ]:
df = pd.read_csv('df.csv')
df = df.drop(columns='Unnamed: 0')

In [ ]:
df = pd.get_dummies(df, columns=['Theme'])

# Удаление столбцов, названия которых отсутствуют в датафрейме
columns_to_drop = [col for col in df.iloc[:,1:].columns if col not in list(theme.theme)]
df = df.drop(columns=columns_to_drop)
df = df[df.sum(axis=1) !=0]

<ipython-input-37-7200e3dda5ed>:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df[df.sum(axis=1) !=0]


In [ ]:
train, test = train_test_split(df, test_size=0.3, random_state=42)
X_train = train.iloc[:,0]
X_test = test.iloc[:,0]
y_train = train.iloc[:,1:-1]
y_test = test.iloc[:,1:-1]

In [5]:
df.to_csv('df.csv')

In [ ]:
vectorizer = CountVectorizer(tokenizer=TweetTokenizer().tokenize, analyzer='word')

X = df.iloc[:,0]
y = df.iloc[:,1:]

X = vectorizer.fit_transform(X)

X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# **Реализация модели; подбор гиперпараметров**

In [ ]:
param_grid = {
    # 'loss_function': ['logloss'],
    'learning_rate': [0.03],
    'depth': [3]
}
grid_search = GridSearchCV(CatBoostClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train.iloc[:,13])

Выходные данные были обрезаны до нескольких последних строк (5000).
939:	learn: 0.2152578	total: 16s	remaining: 1.02s
940:	learn: 0.2148722	total: 16s	remaining: 1s
941:	learn: 0.2148037	total: 16s	remaining: 986ms
942:	learn: 0.2146724	total: 16.1s	remaining: 970ms
943:	learn: 0.2143971	total: 16.1s	remaining: 954ms
944:	learn: 0.2142886	total: 16.1s	remaining: 937ms
945:	learn: 0.2141003	total: 16.1s	remaining: 921ms
946:	learn: 0.2138233	total: 16.2s	remaining: 904ms
947:	learn: 0.2136184	total: 16.2s	remaining: 888ms
948:	learn: 0.2134087	total: 16.2s	remaining: 871ms
949:	learn: 0.2133446	total: 16.2s	remaining: 855ms
950:	learn: 0.2132295	total: 16.3s	remaining: 838ms
951:	learn: 0.2131295	total: 16.3s	remaining: 822ms
952:	learn: 0.2130512	total: 16.3s	remaining: 805ms
953:	learn: 0.2126276	total: 16.3s	remaining: 788ms
954:	learn: 0.2124927	total: 16.4s	remaining: 771ms
955:	learn: 0.2123931	total: 16.4s	remaining: 755ms
956:	learn: 0.2122816	total: 16.4s	remaining: 738ms
957:	

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7fa0aef54f70>,
             param_grid={'depth': [3], 'learning_rate': [0.03]})

In [ ]:
# print("Best parameters: ", grid_search.best_params_)
print("F1-score: ", f1_score(y_test.iloc[:,13], grid_search.predict(X_test)))

F1-score:  0.0
